# 📈 PyKwant Tutorial: Equity Options

This notebook introduces the `pykwant.equity` module.

It focuses on pricing **European Options** using the **Black-Scholes-Merton** framework. Consistent with our functional design, the pricing logic is a pure function that takes the Option (data), Market State (Spot, Vol, Curve), and returns the Price.

## 1. Setup and Imports

In [ ]:
import math
from datetime import date

from pykwant import dates, equity, instruments, numerics


# Helper to print financial values
def print_money(label, amount):
    print(f"{label:<20}: {amount:>10.4f}")

## 2. Market Environment

To price an option, we need:

1. **Spot Price**: Current price of the underlying asset ($S_0$).

2. **Volatility**: Annualized standard deviation of returns ($\sigma$).

3. **Risk-Free Rate**: A Yield Curve.

In [6]:
val_date = date(2025, 1, 1)
spot_price = 100.0
volatility = 0.20  # 20%


# Flat 5% Risk-Free Curve
def risk_free_curve(d: date) -> float:
    t = dates.act_365(val_date, d)
    return math.exp(-0.05 * t)

## 3. Instrument Definition

We define a **Call Option** and a **Put Option** with the same parameters (ATM).

In [7]:
expiry = date(2026, 1, 1)  # 1 Year
strike_price = instruments.Money(100.0)

call_opt = instruments.EuropeanOption(
    asset_name="PYK Tech", strike=strike_price, expiry_date=expiry, call_put="call"
)

put_opt = instruments.EuropeanOption(
    asset_name="PYK Tech", strike=strike_price, expiry_date=expiry, call_put="put"
)

print(call_opt)

EuropeanOption(asset_name='PYK Tech', strike=100.0, expiry_date=datetime.date(2026, 1, 1), call_put='call')


## 4. Black-Scholes Pricing

We use the `black_scholes_price` function. Note how it automatically extracts the correct discount factor from the curve.

In [8]:
call_price = equity.black_scholes_price(
    option=call_opt,
    spot=spot_price,
    volatility=volatility,
    curve=risk_free_curve,
    valuation_date=val_date,
)

put_price = equity.black_scholes_price(
    option=put_opt,
    spot=spot_price,
    volatility=volatility,
    curve=risk_free_curve,
    valuation_date=val_date,
)

print("--- Option Valuation ---")
print_money("Call Price", call_price)
print_money("Put Price", put_price)

--- Option Valuation ---
Call Price          :    10.4506
Put Price           :     5.5735


## 5. Put-Call Parity Check

A fundamental no-arbitrage relationship states:

$$C - P = S - K \cdot e^{-rT}$$

Let's verify if our model holds this truth.

In [9]:
# 1. LHS: Call - Put
lhs = call_price - put_price

# 2. RHS: Spot - PV(Strike)
df = risk_free_curve(expiry)
rhs = spot_price - (strike_price * df)

print("\n--- Put-Call Parity ---")
print_money("C - P", lhs)
print_money("S - K*DF", rhs)

assert abs(lhs - rhs) < 1e-9
print("✅ Parity holds.")


--- Put-Call Parity ---
C - P               :     4.8771
S - K*DF            :     4.8771
✅ Parity holds.


6. Calculating Greeks (Functional Approach)

PyKwant doesn't hardcode formulas for Delta, Gamma, or Vega. Instead, we calculate them using **Numerical Differentiation**.

Since our pricing is a pure function `f(Spot, Vol, ...) -> Price`, we can easily differentiate it with respect to any input.

### Delta ($\Delta$): Sensitivity to Spot Price

$$\Delta \approx \frac{P(S+\epsilon) - P(S-\epsilon)}{2\epsilon}$$

In [10]:
print("\n--- Greeks (Numerical) ---")

epsilon = 0.01


# Create a closure for Price(S)
def price_vs_spot(s: float) -> float:
    return equity.black_scholes_price(call_opt, s, volatility, risk_free_curve, val_date)


# Calculate derivatives
delta_fn = numerics.numerical_derivative(price_vs_spot, h=epsilon)
gamma_fn = numerics.numerical_derivative(delta_fn, h=epsilon)

delta = delta_fn(spot_price)
gamma = gamma_fn(spot_price)

print_money("Delta", delta)  # Should be ~0.63 for ATM Call
print_money("Gamma", gamma)


--- Greeks (Numerical) ---
Delta               :     0.6368
Gamma               :     0.0188


### Vega ($\nu$): Sensitivity to Volatility

In [11]:
# Create a closure for Price(Vol)
def price_vs_vol(v: float) -> float:
    return equity.black_scholes_price(call_opt, spot_price, v, risk_free_curve, val_date)


vega_fn = numerics.numerical_derivative(price_vs_vol, h=1e-4)
vega = vega_fn(volatility)

# Vega is usually quoted for a 1% change (0.01)
print_money("Vega (1%)", vega * 0.01)

Vega (1%)           :     0.3752


## 7. Implied Volatility Solver

A classic problem: Given a market price, what is the **Implied Volatility**?

This is a root-finding problem: Solve for $\sigma$ such that $Price(\sigma) - MarketPrice = 0$. We reuse `numerics.newton_solve` for this.

In [ ]:
target_price = 12.50  # Suppose market price is higher than our model (10.45)

print("\n--- Implied Volatility Solver ---")
print(f"Market Price: {target_price}")


# Define objective function: f(vol) -> Price
def objective_fn(v: float) -> float:
    return equity.black_scholes_price(call_opt, spot_price, v, risk_free_curve, val_date)


# Solve
implied_vol = numerics.newton_solve(
    func=objective_fn,
    target=target_price,
    guess=0.20,  # Start guessing at 20%
    tol=1e-6,
)

print(f"Implied Vol:  {implied_vol:.2%}")

# Verify
check_price = equity.black_scholes_price(
    call_opt, spot_price, implied_vol, risk_free_curve, val_date
)
print(f"Check Price:  {check_price:.4f}")


--- Implied Volatility Solver ---
Market Price: 12.5
Implied Vol:  25.43%
Check Price:  12.5000
